In [ ]:
# =========================
# model_v0 (LB-rule aware)
# - Nonzero-only SMAPE 검증/선택
# - 업장 가중 프록시(담하/미라시아 ↑)
# - Masked MSE(실제=0 가중↓) + weight_decay + grad clip
# - CosineAnnealing + warmup
# - month_sin/cos + inv_d2h + lag_7(feature)
# - 담하/미라시아 전용 파인튜닝 & 캘리브레이션 계수 저장
# - 예측 후 상위 퍼센타일 캡 + 하한 결합(min=1, iqr_lower)
# =========================
import os, random, glob, re
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from tqdm import tqdm
import matplotlib.pyplot as plt
from korean_lunar_calendar import KoreanLunarCalendar

plt.rcParams['font.family'] = 'AppleGothic'  # macOS
# -------------------------
# Global hyperparameters
# -------------------------
LOOKBACK, PREDICT, BATCH_SIZE, EPOCHS = 28, 7, 16, 30
DEVICE = torch.device("cpu")
MONTH_SCALE = 12
MIN_SEQUENCE_COUNT = 10

# 드롭아웃 파라미터(권장 범위 반영)
EMB_DROPOUT = 0.00
LSTM_DROPOUT = 0.05     # num_layers>1에서만 레이어 간 적용
HEAD_DROPOUT = 0.10

# 최적화/정규화
LR = 1e-3
WEIGHT_DECAY = 1e-4
GRAD_CLIP = 1.0
WARMUP_EPOCHS = 3

# 검증/선택에서 업장 가중(리더보드 프록시)
STORE_WEIGHT_PROXY = {'담하': 2.0, '미라시아': 2.0}  # others=1.0

# 담하/미라시아 전용 파인튜닝 epoch (공통 학습 후 추가)
FT_EPOCHS_HIGH_WEIGHT_STORE = 6

# 예측 후 후처리 퍼센타일 캡
UPPER_CAP_PCT = 99.0   # 비영일 기준 상위 99% 캡
# -------------------------
# Utils & seed
# -------------------------
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)
# ---------------------------
# EarlyStopping (for PyTorch)
# ---------------------------
class EarlyStopping:
    """
    - mode='min': metric이 감소할 때 개선으로 판단
    - patience: 개선 없을 때 몇 epoch 기다릴지
    - min_delta: 개선으로 인정할 최소 변화량
    - restore_best: 종료 시 best_state로 자동 복원
    """
    def __init__(self, mode='min', patience=5, min_delta=0.0, restore_best=True):
        assert mode in ('min', 'max')
        self.mode = mode
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best = restore_best

        self.best = None
        self.best_state = None
        self.num_bad = 0
        self.should_stop = False

    def _is_better(self, score):
        if self.best is None:
            return True
        if self.mode == 'min':
            return (self.best - score) > self.min_delta
        else:
            return (score - self.best) > self.min_delta

    def step(self, score, model):
        if self._is_better(score):
            self.best = score
            # CPU 텐서로 저장 (GPU면 .cpu() 호출)
            self.best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            self.num_bad = 0
        else:
            self.num_bad += 1
            if self.num_bad >= self.patience:
                self.should_stop = True

    def restore(self, model):
        if self.restore_best and self.best_state is not None:
            model.load_state_dict(self.best_state)
# -------------------------
# Holiday utilities
# -------------------------
def get_lunar_to_solar(years, lunar_month, lunar_day, span=1):
    calendar = KoreanLunarCalendar()
    dates = []
    for year in years:
        for offset in range(-span, span+1):
            try:
                calendar.setLunar(year, lunar_month, lunar_day + offset, False)
                dates.append(calendar.SolarIsoFormat())
            except:
                pass
    return dates

years = [2023, 2024, 2025]  # 데이터 기간에 맞춰 조정
lunar_solar_dates = []
lunar_solar_dates += get_lunar_to_solar(years, 1, 1, span=1)   # 설날 ±1
lunar_solar_dates += get_lunar_to_solar(years, 8, 15, span=1)  # 추석 ±1

solar_md_holidays = [
    (1,1),(3,1),(5,5),(6,6),(8,15),(10,3),(10,9),(12,25)
]

def build_holiday_set():
    # 음/양력 통합 set
    holo = set(pd.to_datetime(lunar_solar_dates))
    for y in years:
        for m, d in solar_md_holidays:
            holo.add(pd.to_datetime(f"{y}-{m:02d}-{d:02d}"))
    return holo

HOLIDAY_SET = build_holiday_set()

def days_to_nearest_holiday(dt: pd.Timestamp) -> int:
    # 날짜 거리 연산 효율성 위해 근접치만 계산(작은 범위이면 충분)
    # 여기선 ±60일 창에서 근사
    window = pd.date_range(dt - pd.Timedelta(days=60), dt + pd.Timedelta(days=60))
    cands = np.array([abs((h - dt).days) for h in HOLIDAY_SET if h in set(window)])
    return int(cands.min() if len(cands) else 60)

def generate_combined_holiday_list(df, solar_md_list, lunar_solar_list):
    df = df.copy()
    df['영업일자'] = pd.to_datetime(df['영업일자'])
    df['is_solar_holiday'] = df['영업일자'].apply(lambda x: (x.month, x.day) in solar_md_list)
    lunar_set = set(pd.to_datetime(lunar_solar_list))
    df['is_lunar_holiday'] = df['영업일자'].isin(lunar_set)
    df['is_holiday'] = (df['is_solar_holiday'] | df['is_lunar_holiday']).astype(int)
    return df.drop(columns=['is_solar_holiday','is_lunar_holiday'])
# -------------------------
# Preprocess helpers
# -------------------------
def remove_leading_zeros_before_sales(df, min_zero_days=90):
    """매출 시작 전 연속 0이 일정 기간 이상이면 그 전 구간 제거"""
    sales_started = df['매출수량'] > 0
    if not sales_started.any():
        return df
    first_sale_idx = sales_started.idxmax()
    df_before = df.loc[:first_sale_idx-1]
    if len(df_before) >= min_zero_days and (df_before['매출수량'] == 0).all():
        return df.loc[first_sale_idx:]
    return df

def filter_all_menus_by_leading_zeros(train_df, min_zero_days=90):
    return (train_df.groupby('영업장명_메뉴명')
            .apply(lambda g: remove_leading_zeros_before_sales(g, min_zero_days))
            .reset_index(drop=True))

def clip_iqr(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    upper = q3 + 1.5*iqr
    return np.clip(series, None, upper)

def compute_iqr_lower_bounds(train_df):
    lower_bounds = {}
    for menu, group in train_df.groupby('영업장명_메뉴명'):
        q1 = group['매출수량'].quantile(0.25)
        q3 = group['매출수량'].quantile(0.75)
        iqr = q3 - q1
        lower = max(q1 - 1.5*iqr, 0)
        key = menu[0] if isinstance(menu, tuple) else menu
        lower_bounds[key] = lower
    return lower_bounds

def get_store_from_menu(menu_str: str) -> str:
    # '영업장명_메뉴명' 형태에서 앞부분만 취득
    if '_' in menu_str:
        return menu_str.split('_', 1)[0]
    return menu_str
# -------------------------
# Metrics (LB rule aware)
# -------------------------
def smape_nonzero_np(y_true, y_pred, eps=1e-6):
    # 실제 0 제외
    mask = (np.abs(y_true) > 0)
    if mask.sum() == 0:
        return 0.0
    t = y_true[mask]; p = y_pred[mask]
    denom = (np.abs(t) + np.abs(p) + eps)
    return 200.0 * np.mean(np.abs(t - p) / denom)

def weighted_store_smape(val_df: pd.DataFrame) -> float:
    """열: ['영업장명','y_true','y_pred'] 필요"""
    score, wsum = 0.0, 0.0
    for s, g in val_df.groupby('영업장명'):
        w = STORE_WEIGHT_PROXY.get(s, 1.0)
        sc = smape_nonzero_np(g['y_true'].values, g['y_pred'].values)
        score += w * sc; wsum += w
    return score/wsum if wsum > 0 else 0.0
# -------------------------
# Model
# -------------------------
class MultiEmbeddingLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=3,
                 output_dim=7, num_weekdays=7, weekday_embed_dim=3,
                 num_seasons=4, season_embed_dim=2,
                 emb_dropout=0.0, lstm_dropout=0.1, head_dropout=0.1):
        super().__init__()
        # 범주형 임베딩
        self.weekday_embedding = nn.Embedding(num_weekdays, weekday_embed_dim)
        self.season_embedding = nn.Embedding(num_seasons, season_embed_dim)
        # 임베딩 정규화
        self.weekday_norm = nn.LayerNorm(weekday_embed_dim)
        self.season_norm  = nn.LayerNorm(season_embed_dim)
        # 임베딩+수치 concat 후 dropout (정보 보존 위해 소량)
        self.emb_dropout = nn.Dropout(emb_dropout)

        total_input_dim = input_dim + weekday_embed_dim + season_embed_dim

        # LSTM (num_layers>1일 때 레이어 사이 dropout 동작)
        self.lstm = nn.LSTM(total_input_dim, hidden_dim, num_layers,
                            batch_first=True, dropout=(lstm_dropout if num_layers>1 else 0.0))
        # LSTM 출력 정규화 + head dropout
        self.ln = nn.LayerNorm(hidden_dim)
        self.head = nn.Sequential(
            nn.Dropout(head_dropout),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x, weekday_ids, season_ids):
        # x: (B,T,input_dim), weekday_ids/season_ids: (B,T)
        wd = self.weekday_norm(self.weekday_embedding(weekday_ids))
        ss = self.season_norm(self.season_embedding(season_ids))
        x = self.emb_dropout(torch.cat([x, wd, ss], dim=-1))
        out, _ = self.lstm(x)               # (B,T,H)
        out = self.ln(out[:, -1, :])        # 마지막 step 사용
        return self.head(out)               # (B,7)
# -------------------------
# Loss (masked MSE: 실제=0 가중↓)
# -------------------------
def masked_mse(pred, target, zero_weight=1.0):
    nz = (target > 0).float()
    w = nz + (1 - nz) * zero_weight
    return torch.mean(w * (pred - target) ** 2)
    # -------------------------
# Loss/Metric Visualization
# ---------------------------
class LossTracker:
    """
    - train_mse: (epoch별) 학습 손실(MSE 또는 masked MSE)  ※ 스케일된 공간
    - val_mse:   (epoch별) 검증 MSE(선택)
    - val_smape: (epoch별) 검증 SMAPE(리더보드 규칙; nonzero/weighted)
    - lr:        (epoch별) 학습률
    """
    def __init__(self, smooth_k: int = 5, out_dir: str = "./loss_plots"):
        self.train_mse, self.val_mse, self.val_smape, self.lr = [], [], [], []
        self.smooth_k = smooth_k
        self.out_dir = out_dir
        os.makedirs(out_dir, exist_ok=True)

    def log_train(self, train_mse_epoch: float, lr_epoch: float = None):
        self.train_mse.append(float(train_mse_epoch))
        if lr_epoch is not None: self.lr.append(float(lr_epoch))

    def log_val(self, val_mse_epoch: float = None, val_smape_epoch: float = None):
        if val_mse_epoch is not None: self.val_mse.append(float(val_mse_epoch))
        if val_smape_epoch is not None: self.val_smape.append(float(val_smape_epoch))

    @staticmethod
    def _smooth(y, k):
        if y is None or len(y) == 0: return None
        k = max(1, min(k, len(y)))
        w = np.ones(k) / k
        return np.convolve(np.array(y, dtype=float), w, mode="same")

    def _best_epoch_by_smape(self):
        return int(np.argmin(self.val_smape)) if len(self.val_smape) else None

    def plot(self, title: str, save: bool = True, show: bool = False):
        safe = re.sub(r"[^\w\-_.]", "_", title)
        epochs = np.arange(1, len(self.train_mse) + 1)

        # --- Fig 1: Train/Val MSE (+ LR as secondary axis)
        plt.figure(figsize=(8,5))
        plt.plot(epochs, self.train_mse, label="Train MSE", linewidth=1.5)
        if len(self.val_mse):
            # val_mse 길이를 epoch 인덱스에 맞춰 표시
            val_x = epochs[-len(self.val_mse):]
            plt.plot(val_x, self.val_mse, label="Val MSE", linewidth=1.5)
        plt.xlabel("Epoch"); plt.ylabel("MSE (scaled)")
        plt.title(f"[{title}] Train vs Val MSE")
        plt.grid(True); plt.legend(loc="upper right")

        # secondary y-axis: learning rate
        if len(self.lr):
            ax2 = plt.gca().twinx()
            lr_x = epochs[:len(self.lr)]
            ax2.plot(lr_x, self.lr, linestyle="--", alpha=0.6, label="LR")
            ax2.set_ylabel("Learning Rate")
            # 공동 범례
            lines = plt.gca().get_lines() + ax2.get_lines()
            labels = [l.get_label() for l in lines]
            plt.legend(lines, labels, loc="upper center")

        if save:
            plt.savefig(os.path.join(self.out_dir, f"{safe}_mse.png"), bbox_inches="tight")
        if show: plt.show()
        plt.close()

        # --- Fig 2: Validation SMAPE (원 스케일, %). 원곡선 + 스무딩
        if len(self.val_smape):
            val_x = epochs[-len(self.val_smape):]
            sm = self._smooth(self.val_smape, self.smooth_k)
            best_ep_rel = self._best_epoch_by_smape()
            best_ep_abs = val_x[best_ep_rel] if best_ep_rel is not None else None

            plt.figure(figsize=(8,5))
            plt.plot(val_x, self.val_smape, label="Val SMAPE", linewidth=1.5)
            if sm is not None: plt.plot(val_x, sm, label=f"Smoothed(k={self.smooth_k})", linestyle="--")
            if best_ep_abs is not None:
                best_val = self.val_smape[best_ep_rel]
                plt.axvline(best_ep_abs, color="gray", linestyle=":", alpha=0.8)
                plt.annotate(f"best {best_val:.2f}",
                             xy=(best_ep_abs, best_val),
                             xytext=(best_ep_abs+0.5, best_val+2),
                             arrowprops=dict(arrowstyle="->", alpha=0.7))
            plt.xlabel("Epoch"); plt.ylabel("SMAPE (%)")
            plt.title(f"[{title}] Validation SMAPE (nonzero, weighted)")
            plt.grid(True); plt.legend(loc="upper right")
            if save:
                plt.savefig(os.path.join(self.out_dir, f"{safe}_smape.png"), bbox_inches="tight")
            if show: plt.show()
            plt.close()

        # --- Fig 3(옵션): 한눈 요약 카드 (마지막 값/최저값)
        if len(self.val_smape):
            msg = (f"Last SMAPE: {self.val_smape[-1]:.3f} | "
                   f"Best SMAPE: {min(self.val_smape):.3f} (ep {best_ep_abs}) | "
                   f"Train MSE(last): {self.train_mse[-1]:.4f}" )
            print(f"[Summary] {title} -> {msg}")

# -------------------------
# Train
# -------------------------
def train_lstm(train_df, use_validation=True):
    """
    - 검증/선택: weighted_store_smape (실제 0 제외) 기준 EarlyStopping
    - Optim: Adam + weight_decay, grad clip, CosineAnnealing, warmup
    - Feature: month_sin/cos, inv_d2h, lag_7
    - 담하/미라시아: 추가 파인튜닝 및 캘리브레이션 계수 저장
    - 시각화: LossTracker로 Train MSE / Val MSE / Val SMAPE / LR 저장 및 출력
    """
    trained_models = {}
    lower_bound_dict = compute_iqr_lower_bounds(train_df)  # 메뉴별 하한
    upper_cap_dict = {}

    for store_menu, group in tqdm(train_df.groupby(['영업장명_메뉴명']), desc='Training LSTM'):
        key = store_menu[0] if isinstance(store_menu, tuple) else store_menu
        store_name = get_store_from_menu(key)

        # ====== 기본 파생 ======
        g = group.sort_values('영업일자').copy()
        g['영업일자'] = pd.to_datetime(g['영업일자'])
        g['weekday']  = g['영업일자'].dt.dayofweek.astype(int)
        g['month']    = g['영업일자'].dt.month
        g['season']   = g['month'].map({12:0,1:0,2:0, 3:1,4:1,5:1, 6:2,7:2,8:2, 9:3,10:3,11:3}).astype(int)

        # 휴일 0/1 + 근접도
        g = generate_combined_holiday_list(g, solar_md_holidays, lunar_solar_dates)
        g['d2h'] = g['영업일자'].apply(days_to_nearest_holiday).clip(0, 7)
        g['inv_d2h'] = 1.0 / (g['d2h'] + 1.0)

        # 사이클릭 month
        g['month_sin'] = np.sin(2*np.pi*g['month']/12)
        g['month_cos'] = np.cos(2*np.pi*g['month']/12)

        # 타깃 및 통계 특성
        g['clipped_SQ'] = clip_iqr(g['매출수량'])
        g['rolling_mean_7'] = g['clipped_SQ'].rolling(window=7, min_periods=1).mean()
        g['delta'] = g['clipped_SQ'].diff().fillna(0)
        g['lag_7'] = g['clipped_SQ'].shift(7).fillna(method='bfill')

        if len(g) < LOOKBACK + PREDICT + MIN_SEQUENCE_COUNT:
            continue

        # ====== 스케일러 ======
        scaler_main = MinMaxScaler()   # ['clipped_SQ','rolling_mean_7','lag_7']
        scaler_delta = MinMaxScaler()  # ['delta']

        g[['clipped_SQ','rolling_mean_7','lag_7']] = scaler_main.fit_transform(
            g[['clipped_SQ','rolling_mean_7','lag_7']]
        )
        g[['delta_scaled']] = scaler_delta.fit_transform(g[['delta']])

        # 역변환용 통계(첫 특성 = clipped_SQ)
        dmin = scaler_main.data_min_[0]
        drng = scaler_main.data_range_[0]

        # 상한 캡(비영일 기준 99%)
        y_all_inv = (g['clipped_SQ'].values.reshape(-1,1) * drng + dmin).ravel()
        upper_cap = np.percentile(y_all_inv[y_all_inv > 0], UPPER_CAP_PCT) if (y_all_inv > 0).any() else np.inf
        upper_cap_dict[key] = upper_cap

        # ====== 시퀀스 구성 ======
        features = ['clipped_SQ','rolling_mean_7','delta_scaled',
                    'month_sin','month_cos','is_holiday','inv_d2h','lag_7']
        vals = g[features].values

        X, y, wd_seqs, ss_seqs, dates = [], [], [], [], []
        for i in range(len(g) - LOOKBACK - PREDICT + 1):
            X.append(vals[i:i+LOOKBACK])
            y.append(g['clipped_SQ'].values[i+LOOKBACK:i+LOOKBACK+PREDICT])
            wd_seqs.append(g['weekday'].values[i:i+LOOKBACK])
            ss_seqs.append(g['season'].values[i:i+LOOKBACK])
            dates.append(g['영업일자'].values[i+LOOKBACK:i+LOOKBACK+PREDICT])

        X = torch.tensor(np.array(X)).float()
        y = torch.tensor(np.array(y)).float()
        wd_seqs = torch.tensor(np.array(wd_seqs)).long()
        ss_seqs = torch.tensor(np.array(ss_seqs)).long()

        if use_validation:
            split_idx = int(len(X) * 0.8)
            X_tr, X_val = X[:split_idx], X[split_idx:]
            y_tr, y_val = y[:split_idx], y[split_idx:]
            wd_tr, wd_val = wd_seqs[:split_idx], wd_seqs[split_idx:]
            ss_tr, ss_val = ss_seqs[:split_idx], ss_seqs[split_idx:]
            dates_val = np.array(dates[split_idx:])
        else:
            X_tr, y_tr, wd_tr, ss_tr = X, y, wd_seqs, ss_seqs
            X_val = y_val = wd_val = ss_val = None
            dates_val = None

        X_tr, y_tr = X_tr.to(DEVICE), y_tr.to(DEVICE)
        wd_tr, ss_tr = wd_tr.to(DEVICE), ss_tr.to(DEVICE)
        if use_validation:
            X_val, y_val = X_val.to(DEVICE), y_val.to(DEVICE)
            wd_val, ss_val = wd_val.to(DEVICE), ss_val.to(DEVICE)

        # ====== 모델/최적화/트래커/얼리스탑 ======
        model = MultiEmbeddingLSTM(
            input_dim=len(features), output_dim=PREDICT,
            emb_dropout=EMB_DROPOUT, lstm_dropout=LSTM_DROPOUT, head_dropout=HEAD_DROPOUT
        ).to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
        criterion = masked_mse
        mse_criterion = nn.MSELoss()

        tracker = LossTracker(smooth_k=5, out_dir="./loss_plots")
        es = EarlyStopping(mode='min', patience=8, min_delta=0.1, restore_best=True)

        base_lr = LR
        def set_warmup_lr(epoch):
            if epoch < WARMUP_EPOCHS:
                for pg in optimizer.param_groups:
                    pg['lr'] = base_lr * (epoch + 1) / WARMUP_EPOCHS

        # ====== 학습 루프 ======
        for epoch in range(EPOCHS):
            model.train()
            set_warmup_lr(epoch)
            total = 0.0
            idx = torch.randperm(len(X_tr))
            for i in range(0, len(X_tr), BATCH_SIZE):
                b = idx[i:i+BATCH_SIZE]
                xb, yb = X_tr[b], y_tr[b]
                wdb, ssb = wd_tr[b], ss_tr[b]
                pred = model(xb, wdb, ssb)
                loss = criterion(pred, yb)

                optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
                optimizer.step()
                total += loss.item()

            # epoch별 로그
            train_mse_epoch = float(total / (len(X_tr)//BATCH_SIZE + 1))
            current_lr = optimizer.param_groups[0]['lr']
            tracker.log_train(train_mse_epoch, current_lr)

            # ---- 검증 (MSE + SMAPE) ----
            if use_validation:
                model.eval()
                with torch.no_grad():
                    pv = model(X_val, wd_val, ss_val)
                    val_mse_epoch = mse_criterion(pv, y_val).item()

                    pv_np = pv.cpu().numpy(); tv_np = y_val.cpu().numpy()
                    pv_inv = pv_np * drng + dmin
                    tv_inv = tv_np * drng + dmin

                    # 업장 가중 SMAPE
                    rows = []
                    for k_idx in range(len(pv_inv)):
                        for t in range(PREDICT):
                            rows.append([store_name, tv_inv[k_idx][t], pv_inv[k_idx][t]])
                    val_df = pd.DataFrame(rows, columns=['영업장명','y_true','y_pred'])
                    val_smape_epoch = weighted_store_smape(val_df)

                tracker.log_val(val_mse_epoch=val_mse_epoch, val_smape_epoch=val_smape_epoch)

                # ---- EarlyStopping 갱신 ----
                es.step(val_smape_epoch, model)
                if es.should_stop:
                    # print(f"[ES] stop at epoch {epoch+1}, best={es.best:.3f}")
                    break

            scheduler.step()

        # best로 복원
        if use_validation:
            es.restore(model)

        # ====== 고가중치 업장 추가 파인튜닝(옵션) ======
        if STORE_WEIGHT_PROXY.get(store_name, 1.0) > 1.0 and FT_EPOCHS_HIGH_WEIGHT_STORE > 0:
            model.train()
            for _ in range(FT_EPOCHS_HIGH_WEIGHT_STORE):
                total = 0.0
                idx = torch.randperm(len(X_tr))
                for i in range(0, len(X_tr), BATCH_SIZE):
                    b = idx[i:i+BATCH_SIZE]
                    xb, yb = X_tr[b], y_tr[b]
                    wdb, ssb = wd_tr[b], ss_tr[b]
                    pred = model(xb, wdb, ssb)
                    loss = criterion(pred, yb)
                    optimizer.zero_grad()
                    loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
                    optimizer.step()

        # ====== 캘리브레이션 계수(a) ======
        calib_a = 1.0
        if use_validation:
            with torch.no_grad():
                pv = model(X_val, wd_val, ss_val).cpu().numpy()
            pv_inv = pv * drng + dmin
            tv_inv = y_val.cpu().numpy() * drng + dmin
            mask = tv_inv > 0
            if mask.sum() > 0:
                ratio = (tv_inv[mask] + 1e-6) / (pv_inv[mask] + 1e-6)
                calib_a = float(np.median(ratio))

        # ====== 그래프 저장 ======
        tracker.plot(title=key, save=True, show=False)

        # ====== 결과 저장 ======
        trained_models[store_menu] = {
            'model': model.eval(),
            'scaler_main': scaler_main,
            'scaler_delta': scaler_delta,
            'last_sequence': {
                'X': vals[-LOOKBACK:],  # 스케일 적용 상태
                'weekday': g['weekday'].values[-LOOKBACK:],
                'season':  g['season'].values[-LOOKBACK:]
            },
            'lower_bound': max(1, lower_bound_dict.get(key, 0.0)),
            'upper_cap': upper_cap,
            'calib_a': calib_a,
            'store_name': store_name
        }

    return trained_models


# -------------------------
# Predict
# -------------------------
def predict_lstm(test_df, trained_models, test_prefix: str):
    """
    - 최근 LOOKBACK 윈도우 구성 시 train에서 저장한 스케일러로 transform
    - 예측값 복원: scaler_main의 첫 컬럼 통계로만 역변환
    - 하한/상한 캡 + 업장별 캘리브레이션(a*pred) 후 제출
    """
    results = []
    for store_menu, store_test in test_df.groupby(['영업장명_메뉴명']):
        key = store_menu
        if key not in trained_models:
            continue

        pack = trained_models[key]
        model = pack['model']
        scaler_main = pack['scaler_main']
        scaler_delta = pack['scaler_delta']
        lower_bound = pack['lower_bound']
        upper_cap = pack['upper_cap']
        calib_a = pack['calib_a']
        store_name = pack['store_name']

        st = store_test.sort_values('영업일자').copy()
        st['영업일자'] = pd.to_datetime(st['영업일자'])
        st['weekday']  = st['영업일자'].dt.dayofweek.astype(int)
        st['month']    = st['영업일자'].dt.month
        st['season']   = st['month'].map({12:0,1:0,2:0, 3:1,4:1,5:1, 6:2,7:2,8:2, 9:3,10:3,11:3}).astype(int)
        st = generate_combined_holiday_list(st, solar_md_holidays, lunar_solar_dates)
        st['d2h'] = st['영업일자'].apply(days_to_nearest_holiday).clip(0,7)
        st['inv_d2h'] = 1.0/(st['d2h'] + 1.0)
        st['clipped_SQ'] = clip_iqr(st['매출수량'])
        st['rolling_mean_7'] = st['clipped_SQ'].rolling(window=7, min_periods=1).mean()
        st['delta'] = st['clipped_SQ'].diff().fillna(0)
        st['lag_7'] = st['clipped_SQ'].shift(7).fillna(method='bfill')
        st['month_sin'] = np.sin(2*np.pi*st['month']/12)
        st['month_cos'] = np.cos(2*np.pi*st['month']/12)

        # 최근 LOOKBACK 구성
        if len(st) < LOOKBACK:
            last = pack['last_sequence']
            x_input = torch.tensor([last['X']]).float().to(DEVICE)
            wd_seq = torch.tensor([last['weekday']]).long().to(DEVICE)
            ss_seq = torch.tensor([last['season']]).long().to(DEVICE)
        else:
            recent = st.iloc[-LOOKBACK:].copy()
            # 스케일 transform (train에서 fit한 scaler 기준)
            recent[['clipped_SQ','rolling_mean_7','lag_7']] = scaler_main.transform(
                recent[['clipped_SQ','rolling_mean_7','lag_7']]
            )
            recent[['delta_scaled']] = scaler_delta.transform(recent[['delta']])

            feature_cols = ['clipped_SQ','rolling_mean_7','delta_scaled',
                            'month_sin','month_cos','is_holiday','inv_d2h','lag_7']
            x_input = torch.tensor([recent[feature_cols].values]).float().to(DEVICE)
            wd_seq = torch.tensor([recent['weekday'].values]).long().to(DEVICE)
            ss_seq = torch.tensor([recent['season'].values]).long().to(DEVICE)

        with torch.no_grad():
            pred_scaled = model(x_input, wd_seq, ss_seq).squeeze().cpu().numpy()

        # 첫 컬럼 역변환: scaler_main의 첫 특성(clipped_SQ) 통계로 복원
        dmin = pack['scaler_main'].data_min_[0]
        drng = pack['scaler_main'].data_range_[0]
        restored = pred_scaled * drng + dmin

        # 업장별 캘리브레이션(a*pred) + 하한/상한 클리핑
        restored = calib_a * restored
        restored = np.clip(restored, lower_bound, upper_cap)

        # 제출 형식화
        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(PREDICT)]
        for d, val in zip(pred_dates, restored):
            results.append({'영업일자': d, '영업장명_메뉴명': store_menu[0], '매출수량': float(val)})

    return pd.DataFrame(results)
# -------------------------
# Submission converter
# -------------------------
def convert_to_submission_format(pred_df: pd.DataFrame, sample_submission: pd.DataFrame):
    pred_dict = dict(zip(zip(pred_df['영업일자'], pred_df['영업장명_메뉴명']),
                         pred_df['매출수량']))
    final_df = sample_submission.copy()
    for row_idx in final_df.index:
        date = final_df.loc[row_idx, '영업일자']
        for col in final_df.columns[1:]:
            final_df.loc[row_idx, col] = pred_dict.get((date, col), 0)
    return final_df
# -------------------------
# Data load & preprocess (train)
# -------------------------
# 사용 예:
# train = pd.read_csv('./train/train.csv')
# train = generate_combined_holiday_list(train, solar_md_holidays, lunar_solar_dates)
# train = filter_all_menus_by_leading_zeros(train, min_zero_days=90)
# trained_models = train_lstm(train, use_validation=True)

# -------------------------
# Inference (test)
# -------------------------
# 사용 예:
# for path in sorted(glob.glob('./test/TEST_*.csv')):
#     test_prefix = re.search(r'(TEST_\d+)', os.path.basename(path)).group(1)
#     test_df = pd.read_csv(path)
#     pred_df = predict_lstm(test_df, trained_models, test_prefix)
#     ...


In [21]:
train = pd.read_csv('./train/train.csv')
train = generate_combined_holiday_list(train, solar_md_holidays, lunar_solar_dates)
train = filter_all_menus_by_leading_zeros(train, min_zero_days=90)
trained_models = train_lstm(train, use_validation=True)

/var/folders/r4/sdnz117n6pl22zr9jhhv5vbm0000gn/T/ipykernel_62654/1456909957.py:169: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: remove_leading_zeros_before_sales(g, min_zero_days))
Training LSTM:   0%|          | 0/193 [00:00<?, ?it/s]/var/folders/r4/sdnz117n6pl22zr9jhhv5vbm0000gn/T/ipykernel_62654/1456909957.py:396: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  g['lag_7'] = g['clipped_SQ'].shift(7).fillna(method='bfill')
Training LSTM:   1%|          | 1/193 [00:07<22:49,  7.13s/it]

[Summary] 느티나무 셀프BBQ_1인 수저세트 -> Last SMAPE: 68.923 | Best SMAPE: 59.396 (ep 10) | Train MSE(last): 0.0526


/var/folders/r4/sdnz117n6pl22zr9jhhv5vbm0000gn/T/ipykernel_62654/1456909957.py:396: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  g['lag_7'] = g['clipped_SQ'].shift(7).fillna(method='bfill')
Training LSTM:   1%|          | 2/193 [00:13<21:14,  6.67s/it]

[Summary] 느티나무 셀프BBQ_BBQ55(단체) -> Last SMAPE: 38.112 | Best SMAPE: 34.354 (ep 8) | Train MSE(last): 0.0577


/var/folders/r4/sdnz117n6pl22zr9jhhv5vbm0000gn/T/ipykernel_62654/1456909957.py:396: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  g['lag_7'] = g['clipped_SQ'].shift(7).fillna(method='bfill')
Training LSTM:   2%|▏         | 3/193 [00:19<19:58,  6.31s/it]

...

[Summary] 화담숲카페_아메리카노 ICE -> Last SMAPE: 49.486 | Best SMAPE: 45.223 (ep 11) | Train MSE(last): 0.0371


/var/folders/r4/sdnz117n6pl22zr9jhhv5vbm0000gn/T/ipykernel_62654/1456909957.py:396: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  g['lag_7'] = g['clipped_SQ'].shift(7).fillna(method='bfill')
Training LSTM:  99%|█████████▉| 192/193 [43:54<00:05,  5.75s/it]

[Summary] 화담숲카페_카페라떼 ICE -> Last SMAPE: 54.575 | Best SMAPE: 39.821 (ep 3) | Train MSE(last): 0.0511


/var/folders/r4/sdnz117n6pl22zr9jhhv5vbm0000gn/T/ipykernel_62654/1456909957.py:396: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  g['lag_7'] = g['clipped_SQ'].shift(7).fillna(method='bfill')
Training LSTM: 100%|██████████| 193/193 [44:02<00:00, 13.69s/it]

[Summary] 화담숲카페_현미뻥스크림 -> Last SMAPE: 49.598 | Best SMAPE: 42.890 (ep 13) | Train MSE(last): 0.0260


In [22]:
all_preds = []

# 모든 test_*.csv 순회
test_files = sorted(glob.glob('./test/TEST_*.csv'))
df = pd.read_csv('./train/train.csv')
for path in test_files:
    test_df = pd.read_csv(path)
    # 파일명에서 접두어 추출 (예: TEST_00)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    pred_df = predict_lstm(test_df, trained_models, test_prefix)
    all_preds.append(pred_df)
    
full_pred_df = pd.concat(all_preds, ignore_index=True)

/var/folders/r4/sdnz117n6pl22zr9jhhv5vbm0000gn/T/ipykernel_62654/1456909957.py:617: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  st['lag_7'] = st['clipped_SQ'].shift(7).fillna(method='bfill')
/var/folders/r4/sdnz117n6pl22zr9jhhv5vbm0000gn/T/ipykernel_62654/1456909957.py:637: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  x_input = torch.tensor([recent[feature_cols].values]).float().to(DEVICE)
/var/folders/r4/sdnz117n6pl22zr9jhhv5vbm0000gn/T/ipykernel_62654/1456909957.py:617: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  st['lag_7'] = st['clipped_SQ'].shift(7).fi

In [24]:
sample_submission = pd.read_csv('./sample_submission.csv')
submission = convert_to_submission_format(full_pred_df, sample_submission)
submission.to_csv('./Prediction/model_v0.csv', index=False, encoding='utf-8-sig')
result = pd.read_csv('./Prediction/model_v0.csv')
display(result.head())

/var/folders/r4/sdnz117n6pl22zr9jhhv5vbm0000gn/T/ipykernel_62654/1456909957.py:669: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '8.842390206622419' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  final_df.loc[row_idx, col] = pred_dict.get((date, col), 0)
/var/folders/r4/sdnz117n6pl22zr9jhhv5vbm0000gn/T/ipykernel_62654/1456909957.py:669: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '19.1721001110435' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  final_df.loc[row_idx, col] = pred_dict.get((date, col), 0)
/var/folders/r4/sdnz117n6pl22zr9jhhv5vbm0000gn/T/ipykernel_62654/1456909957.py:669: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '11.557884944015717' has dtyp

,영업일자,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_BBQ55(단체),"느티나무 셀프BBQ_대여료 30,000원","느티나무 셀프BBQ_대여료 60,000원","느티나무 셀프BBQ_대여료 90,000원","느티나무 셀프BBQ_본삼겹 (단품,실내)",느티나무 셀프BBQ_스프라이트 (단체),느티나무 셀프BBQ_신라면,느티나무 셀프BBQ_쌈야채세트,...,화담숲주막_스프라이트,화담숲주막_참살이 막걸리,화담숲주막_찹쌀식혜,화담숲주막_콜라,화담숲주막_해물파전,화담숲카페_메밀미숫가루,화담숲카페_아메리카노 HOT,화담숲카페_아메리카노 ICE,화담숲카페_카페라떼 ICE,화담숲카페_현미뻥스크림
0,TEST_00+1일,8.842390,19.172100,11.557885,1.198290,1.614182,2.500000,1,5.403281,4.361228,...,8.372561,22.756170,23.415746,14.653991,82.520850,20.578683,10.873949,56.779752,10.420427,18.239021
1,TEST_00+2일,7.679180,37.565736,7.412428,2.323247,1.643621,1.777659,1,5.729522,3.574672,...,4.813498,18.488276,19.542077,11.002634,59.007526,40.279306,3.762378,63.032249,12.279731,16.122372
2,TEST_00+3일,6.855104,48.446475,4.300486,1.000000,1.274858,2.068547,1,4.763024,4.761327,...,2.247821,7.625005,5.835684,3.991636,32.335413,19.106218,13.971428,58.179253,17.277071,2.966843
3,TEST_00+4일,8.059332,75.392617,1.710887,1.038149,1.495082,2.500000,1,5.790268,4.150305,...,1.000000,2.827614,1.000000,2.823297,1.000000,43.181086,7.772129,49.371231,9.467475,6.024447
4,TEST_00+5일,8.590896,71.078776,5.770133,1.033821,1.000000,2.160038,1,5.790643,4.073787,...,3.261154,1.000000,1.000000,2.873194,1.369802,22.832903,3.040665,53.916623,8.478695,3.214310
